In [5]:
import pandas as pd
import scipy as sc
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [6]:
data_cleaned = pd.read_csv("data/data_cleaned.csv")
label = data_cleaned["Categorie1"].values

In [7]:
data_cleaned.head()

,Categorie1,Description
0,INFORMATIQUE,batter acer aspir one ion mah noir compatibl b...
1,TELEPHONIE - GPS,coqu rigid bleu lagon alcatel motif drapeau li...
2,TELEPHONIE - GPS,facad coqu cellular lin shckgal minip marqu ag...
3,TELEPHONIE - GPS,coqu meteor tpu nexus
4,TELEPHONIE - GPS,coqu soupl transparent flex motif keep calm an...


In [9]:
features = sc.sparse.load_npz("data/feature_sparse.npz")

In [10]:
features.getrow(0).indices

array([  314,  3258,  4798, 11521, 27233, 32322, 37207, 38199],
      dtype=int32)

## Train test split 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=42)

In [12]:
X_train

<90000x57925 sparse matrix of type '<class 'numpy.int64'>'
	with 955069 stored elements in Compressed Sparse Row format>

In [13]:
X_test

<10000x57925 sparse matrix of type '<class 'numpy.int64'>'
	with 106031 stored elements in Compressed Sparse Row format>

In [14]:
y_train

array(['BIJOUX -  LUNETTES - MONTRES', 'MEUBLE ', 'ELECTROMENAGER', ...,
       'INFORMATIQUE', 'TELEPHONIE - GPS', 'INFORMATIQUE'], dtype=object)

In [15]:
y_test

array(['JOUET (NEW)', 'TELEPHONIE - GPS', 'TELEPHONIE - GPS', ...,
       'TV - VIDEO - SON', 'ADULTE - EROTIQUE ', 'TELEPHONIE - GPS'],
      dtype=object)

## Regression Logistique

In [16]:
lr = LogisticRegression()
#LogisticRegression?

In [17]:
ts = time.time()
lr.fit(X_train, y_train)
te = time.time()
print(te-ts)

/Users/brendanguillouet/anaconda3/envs/INSA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/brendanguillouet/anaconda3/envs/INSA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


46.97428917884827


In [18]:
Y_train_predict = lr.predict(X_train)
lr.score(X_train,y_train)

0.9789444444444444

# Explication

Y_train predict contient les prédictions description par description
ici on voit les prédictions des 3 premiers fichiers.

In [21]:
Y_train_predict[:3]

array(['BIJOUX -  LUNETTES - MONTRES', 'MEUBLE ', 'ELECTROMENAGER'],
      dtype=object)

Vous pouvez retrouver le score a partir de cette prédiction en faisant:
Nombre de descriptions dont la prédiction est juste/Nombre de descriptions total.

Vous voyez que le score est identique à celui calculé par lr.score (qui fait la même chose).

In [26]:
nb_good_prediction = sum(Y_train_predict == y_train)
n_total = len(y_train)
score = nb_good_prediction/n_total
print(score)

0.9789444444444444


Vous pouvez maintenant faire la même chose mais par catégorie seulement. 
Pour cela on se ramène a un prédiction de une catégorie par exemple pour cette catégorie:

In [30]:
#Liste de catégorie unique
unique_catégorie = list(set(y_train))

'JARDIN - PISCINE'

In [39]:
cat_name = 'JARDIN - PISCINE'
Y_train_predict_cat = Y_train_predict==cat_name #P
y_train_cat = y_train==cat_name

nb_good_prediction = sum(Y_train_predict_cat == y_train_cat)
n_total = len(y_train_cat)
score = nb_good_prediction/n_total
print(score)

0.9991444444444444


Le score (accuracy) est très bon mais il est trompeur. Car finalement nous avons beaucoup d'élément qui ne sont PAS de cette catégorie. 
Il faut donc regarder un peu plus en détails d'autre score à partir de la table des éléments croisé.

Vous pouvez voir qu'il y a énormément de VRAI NEGATIF (element prédit faux qui sont effectivement faux) c'est ce qui permet d'avoir un bon score.

In [43]:
A=pd.crosstab(y_train_cat, Y_train_predict_cat, rownames=["Ground True"], colnames=["Prediction"])

Pour avoir une meilleur idée on peut combiner ces quatres score:
* TRUE NEGATIVE : élément faux prédit faux / tout les élement faux
* TRUE POSITIVE : élément vrai prédit vrai / tout les élement vrai
* FALSE NEGATIVE : élément vrai prédit faux / tout les élement vrai
* FALSE POSITIVE :  élément faux prédit vrai/ tout les élement faux

(https://en.wikipedia.org/wiki/F1_score#diagnostic_testing (Partie diagnostic testing)

Le code en bas permet de les calculer (je vous laisse regarder ça)

In [52]:
A/A.sum(axis=1)*100

Prediction,False,True
Ground True,,
False,99.987609,0.899428
True,0.074344,94.603434


## Score par catégorie

### Reduction de X train

In [36]:
nb = 10000
X_small_train = X_train[0:nb]
y_small_train = y_train[0:nb]

In [37]:
ts = time.time()
lr.fit(X_small_train, y_small_train)
te = time.time()
print(te-ts)

2.642327070236206


In [38]:
Y_small_train_predict = lr.predict(X_small_train)
lr.score(X_small_train,y_small_train)

0.989

In [161]:
lr.score(X_test,y_test)

0.9015

#index_gps = lbl_gps.loc[lbl_gps['Categorie1'] == 'TELEPHONIE - GPS'].index
#features_gps = features[v,:]
#lbl_gps = lbl_gps.reset_index(drop=True)


    ### Grosse erreur: on voulait faire la regression logistique par catégorie au lieu d'obtenir le score par categorie

## Comments:
 
Observations : plus on réduit le nombre de produit dans X_train plus notre score de classification est bon sur ce même set. A l'inverse le taux de classification se dégrade sur l'ensemble test. Cohérent car il y a moins de données pour l'apprentissage. On obtient un score de 82,89% pour des set d'apprentissage et de test de 10 000 produits.



 - explorer nb features / ???
 - reg log à recalculer -- done
 - par catégorie ? erreur Ni déséquilibrés, précision par classe
 - accuracy pas la meilleure métrique -- done
 - voir a quelle vitesse on perd l'accuracy -- done
 - réduire X train -- done
 - pas toucher à random_state et test_size / test -- done
 
 